**A few early test analyses of phenocycler downloaded cell data** 

In [ ]:
import numpy as np
import numpy.linalg as linalg
import scipy as scpy
import scipy.stats as stats
from scipy.stats import wishart, multivariate_normal
import pandas as pd
import matplotlib as mplt
import matplotlib.pyplot as plt
import math as math 
import time as tm
import seaborn as sb

cellsdf = pd.read_csv('sample_Data.csv')
print(f'Data read in from file with shape {cellsdf.shape}')


**This plot just shows approximate total cell density over the image***

In [ ]:
fig, ax = plt.subplots()
bns=(22,15)
h,xe,ye,i = ax.hist2d(cellsdf.Xc, cellsdf.Yc, bins=bns)
plt.show()
print(f'Total number of cells is {cellsdf.shape[0]}')
print(f'Boxes are {(xe[bns[0]]-xe[0])/bns[0]:.1f} x {(ye[bns[1]]-ye[0])/bns[1]:.1f} micro m')
print(f'min,max,mean and sd of density (cel/box) {h.min()},{h.max()},{h.mean()},{h.std():.1f}')
he = np.ones(bns)*h.mean()
chisq,p = stats.chisquare(f_obs=np.reshape(h,bns[0]*bns[1]),f_exp=np.reshape(he,bns[0]*bns[1]))
print(f'pvalue for deviation from uniform distribution: {p:0.2f}')

**Descriptive statistics of the cellular stain data**

In [ ]:
cellsdf.describe()

**Histograms of stains and cell areas** and Binarization

In [ ]:
# this bit of code just replaces all the zeros in the data by the min value in the respective col 

small=0.0001
large=100000
dftmp=cellsdf.iloc[:,5:]
dftmp = dftmp.where(dftmp>small, other=large)
mins= dftmp.min(numeric_only=True)
for i in range(0,dftmp.shape[1]):    dftmp.iloc[dftmp.iloc[:,i]>(large-1),i] = mins[i]

# Apply thresholds and convert values to binary based on the top quartile
thresholds = dftmp.quantile(0.75)
print('Before')
print(dftmp.head())
for i in range( dftmp.shape[1]):
    for j in range( dftmp.shape[0]):
        if dftmp.iloc[j,i] > thresholds[i]:
            dftmp.iloc[j,i] = 1.0
        else:
            dftmp.iloc[j,i] = 0.0
print('After')
print(dftmp.head())

**Stain (and area) correlations**

In [ ]:
r,p = stats.spearmanr(cellsdf.iloc[:,5:cellsdf.shape[1]])
df = pd.DataFrame(r, index=cellsdf.columns[5:], columns=cellsdf.columns[5:])
fig,ax = plt.subplots()
sb.heatmap(df,ax=ax, cmap='bwr')
plt.show()


**A quick look at how cells that are strongly positive (top quartile of stain intensity) for these stains are distributed**

In [ ]:

numeric_df = cellsdf.select_dtypes(include=[float, int])
# Now calculate the quantile
thresholds = numeric_df.quantile(0.75)
#  slice the result
thresholds = thresholds[3:]
#thresholds = cellsdf.quantile(0.75)[3:] line not woking
nr=4
nc=4
bns=(22,15)
fig, ax = plt.subplots(nrows=nr,ncols=nc,figsize=(13.0,10.0),layout='constrained')
chi=[]
outs=[]
ncells = []
for i in range(0,thresholds.shape[0]):
    col=i+5
    dfi = cellsdf[cellsdf.iloc[:,col]>thresholds[i]]
    ncells.append(dfi.shape[0])
    r=math.floor(i/nc)
    c=i%nc
    ax[r][c].set_title(cellsdf.columns[col])
    outs.append(ax[r][c].hist2d(dfi.Xc, dfi.Yc, bins=bns))
    expect=np.ones(bns)*outs[i][0].mean()
    chi.append(stats.chisquare(f_obs=np.reshape(outs[i][0],bns[0]*bns[1]),f_exp=np.reshape(expect,bns[0]*bns[1])))
    
plt.show()

for i in range(0,thresholds.shape[0]):
    col=i+5
    print(f'Number of highly expressing (>{thresholds[i]:.2f}) {cellsdf.columns[col]} cells {ncells[i]}')
    print(f'Boxes are {(outs[i][1][bns[0]]-outs[i][1][0])/bns[0]:.1f} x {(outs[i][2][bns[1]]-outs[i][2][0])/bns[1]:.1f} micro m')
    print(f'min,max,mean and sd of density (cells/box) {outs[i][0].min()},{outs[i][0].max()},{outs[i][0].mean():.1f},{outs[i][0].std():.1f}')
    print(f'pvalue for deviation from random distribution: {chi[i][1]:.3f}\n')

**Some exploratory clustering**

Start with a clustermap from seaborn.   

In [ ]:

#now cluster on all stains
cg=sb.clustermap(dftmp, metric='correlation',cmap='bwr',z_score=1, center=0.0, vmin=-2.0, vmax=2.0)


**Process the clusters**

Annoyingly clustermap does not return the cluster info in usable (non graphic) form so need to run the scipy routine it uses again. 

In [ ]:
# clustering above was on col z scores so need these first
means=dftmp.mean(axis=0)
sds= dftmp.std(axis=0)
for i in range(0,dftmp.shape[1]):
    dftmp.iloc[:,i] = (dftmp.iloc[:,i]-means[i])/sds[i]
z=scpy.cluster.hierarchy.linkage(dftmp, method='average', metric='correlation')
print('Done')

**Calinski Habaraz and Silluete score**

In [ ]:
from sklearn.metrics import calinski_harabasz_score, silhouette_score
import matplotlib.pyplot as plt

max_clusters = 14
ch_scores = []
silhouette_scores = []

for nclusters in range(3, max_clusters + 1):  # Start from 3 clusters
    clusters = np.arange(cellsdf.shape[0])
    for i in range(0, cellsdf.shape[0] - nclusters):
        clusters[clusters == z[i, 0]] = cellsdf.shape[0] + i
        clusters[clusters == z[i, 1]] = cellsdf.shape[0] + i

    # Renumber from 1 to nclusters
    nclus = 0
    for i in range(0, clusters.size):
        if clusters[i] < 0:
            continue
        c = clusters[i]
        nclus += 1
        clusters[clusters == c] = -nclus
    clusters = -clusters

    # Work out cluster sizes
    clustersizes = np.zeros(nclus)
    for i in range(0, clusters.size):
        clustersizes[clusters[i] - 1] += 1

    cellsdf['CellType'] = clusters
    dftmp['CellType'] = clusters
    ncelltypes = nclusters

    # Calculate Calinski-Harabasz score
    ch_score = calinski_harabasz_score(dftmp.drop(columns=['CellType']), clusters)
    ch_scores.append(ch_score)

    # Calculate Silhouette score
    silhouette_avg = silhouette_score(dftmp.drop(columns=['CellType']), clusters)
    silhouette_scores.append(silhouette_avg)

# Find the optimal number of clusters
optimal_clusters_Calinski_Harabasz = ch_scores.index(max(ch_scores)) + 3  # +3 because we started from 3 clusters

max_silhouette_score = max(silhouette_scores)
optimal_clusters_Silhouette = silhouette_scores.index(max_silhouette_score) + 3

print(f"Optimal number of clusters based on Calinski-Harabasz score: {optimal_clusters_Calinski_Harabasz}")
print(f"Optimal number of clusters based on Silhouette score: {optimal_clusters_Silhouette}")

# Plot the Calinski-Harabasz and Silhouette scores
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(range(3, max_clusters + 1), ch_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Calinski-Harabasz Score')
plt.title('Calinski-Harabasz Scores for Different Numbers of Clusters')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(3, max_clusters + 1), silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Scores for Different Numbers of Clusters')
plt.grid(True)

plt.tight_layout()
plt.show()




**Process clusters**

In [ ]:
nclusters=5
clusters = np.arange(cellsdf.shape[0])
for i in range(0,cellsdf.shape[0]-nclusters):
    clusters[ clusters==z[i,0] ] = cellsdf.shape[0]+i
    clusters[ clusters==z[i,1] ] = cellsdf.shape[0]+i 

# now renumber from 1 to nclusters
nclus=0
for i in range(0,clusters.size):
    if clusters[i]<0:
        continue
    c=clusters[i]
    nclus += 1
    clusters[clusters==c] = -nclus
clusters = -clusters    

# work out cluster sizes
clustersizes = np.zeros(nclus)
for i in range(0,clusters.size):
    clustersizes[clusters[i]-1] += 1

cellsdf['CellType'] = clusters
dftmp['CellType'] = clusters
ncelltypes = nclusters

print(f'Generated {nclus} clusters')
print('Cluster sizes and centres')

for i in range(0,nclusters):
    print(f'Cluster {i+1} is size {clustersizes[i]}')
    means = dftmp.loc[ dftmp.loc[:,'CellType'] == (i+1), :].mean()
    print(means)




**Distribution of cell types over the image**

In [ ]:
nr=2
nc=3
bns=(22,15)
fig, ax = plt.subplots(nrows=nr,ncols=nc,figsize=(10.0,5.0),layout='constrained')
chi=[]
outs=[]
ncells = []
for i in range(0,ncelltypes):
    dfi = cellsdf[ cellsdf['CellType']==(i+1) ]
    ncells.append(dfi.shape[0])
    r=math.floor(i/nc)
    c=i%nc
    ax[r][c].set_title('Cell Type ' + str(i+1))
    outs.append(ax[r][c].hist2d(dfi.Xc, dfi.Yc, bins=bns))
    expect=np.ones(bns)*outs[i][0].mean()
    chi.append(stats.chisquare(f_obs=np.reshape(outs[i][0],bns[0]*bns[1]),f_exp=np.reshape(expect,bns[0]*bns[1])))
    
plt.show()

for i in range(0,ncelltypes):
    print(f'Number of Cell Type {i+1} cells {ncells[i]}')
    print(f'Boxes are {(outs[i][1][bns[0]]-outs[i][1][0])/bns[0]:.1f} x {(outs[i][2][bns[1]]-outs[i][2][0])/bns[1]:.1f} micro m')
    print(f'min,max,mean and sd of density (cells/box) {outs[i][0].min()},{outs[i][0].max()},{outs[i][0].mean():.1f},{outs[i][0].std():.1f}')
    print(f'pvalue for deviation from random distribution: {chi[i][1]:.3f}\n')

PCA

In [ ]:

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import calinski_harabasz_score, silhouette_score
from umap import UMAP
from scipy.cluster.hierarchy import linkage, dendrogram

dftmp_array = dftmp.values

# always wise to set a random seed 
# so all random
# operations can be repeated 
random_seed = 123
np.random.seed(random_seed)

# reduce to 2 components using PCA
pca_2d = PCA(n_components = 2, random_state=random_seed).fit(dftmp_array)
data_2d = pca_2d.transform(dftmp_array)

# gold standard is to report the amount of variance accounted for in each of the new
# PCA dimensions
variance = pca_2d.explained_variance_ratio_

UMAP

In [ ]:
# Dimensionality reduction through UMAP - a bit more involved than PCA
# but a bit more resilient to imbalance in data
embedder = UMAP(n_components=2, min_dist=0.76, n_neighbors=55).fit(dftmp)
embedding = embedder.transform(dftmp)

In [ ]:
# plot
fig, axes = plt.subplots(2,2,figsize = (10,8))
ax = axes.ravel()
## PCA dimension plots
# start with a simple scatterplot - quite tricky to see though
sb.scatterplot(x = data_2d[:,0], 
                y = data_2d[:,1], 
                # hue = clusters, 
                # palette = 'tab10', 
                ax = ax[0])
# may be easier to see with a kdeplot - kernel density estimation
sb.kdeplot(x = data_2d[:,0], 
                y = data_2d[:,1], 
                # hue = clusters, 
                # palette = 'tab10', 
                ax = ax[1], 
                fill = True, 
                alpha = 0.5) # alpha is level of opacity - need a little transparency to see overlaps

## UMAP embedding plots
sb.scatterplot(x = embedding[:,0], 
                y = embedding[:,1], 
                # hue = clusters, 
                # palette = 'tab10', 
                ax = ax[2])
# may be easier to see with a kdeplot - kernel density estimation
sb.kdeplot(x = embedding[:,0], 
                y = embedding[:,1], 
                # hue = clusters, 
                # palette = 'tab10', 
                ax = ax[3], 
                fill = True, 
                alpha = 0.5) # alpha is level of opacity - need a little transparency to see overlaps

# titles and axes labels
ax[0].set_title('PCA - scatter', fontsize = 12)
ax[1].set_title('PCA - KDE', fontsize = 12)
ax[2].set_title('UMAP - scatter', fontsize = 12)
ax[3].set_title('UMAP - KDE', fontsize = 12)

ax[0].set_xlabel(f'PCA 1 (Explained variance: {str(round(variance[0]*100,2))}%)', fontsize = 12, fontweight = 'bold')
ax[1].set_xlabel(f'PCA 1 (Explained variance: {str(round(variance[0]*100,2))}%)', fontsize = 12, fontweight = 'bold')
ax[2].set_xlabel(f'UMAP 1', fontsize = 12, fontweight = 'bold')
ax[3].set_xlabel(f'UMAP 1', fontsize = 12, fontweight = 'bold')

ax[0].set_ylabel(f'PCA 2 (Explained variance: {str(round(variance[1]*100,2))}%)', fontsize = 12, fontweight = 'bold')
ax[1].set_ylabel(f'PCA 2 (Explained variance: {str(round(variance[1]*100,2))}%)', fontsize = 12, fontweight = 'bold')
ax[2].set_ylabel(f'UMAP 2', fontsize = 12, fontweight = 'bold')
ax[3].set_ylabel(f'UMAP 2', fontsize = 12, fontweight = 'bold')

ax[1].set_xlim([-20,20])
ax[1].set_ylim([-30,30])

fig.tight_layout()

Optimal UMAP parameters 

In [ ]:
from itertools import product
# range of parameters to trial
min_dist_range = [i/100 for i in range(1,100,25)]
n_neighbours_range = list(range(5,105,25))

fig, ax = plt.subplots(len(min_dist_range), len(n_neighbours_range), figsize = (12,12))

# scan through parameter sets and make plots
for i, min_dist in enumerate(min_dist_range):
    for j, n_neighbors in enumerate(n_neighbours_range):
        # make UMAP embedding
        embedding = UMAP(n_components=2, min_dist=min_dist, n_neighbors=n_neighbors).fit_transform(dftmp_array)
        # plot
        sb.scatterplot(x = embedding[:,0], y = embedding[:,1], ax = ax[i][j])
        ax[i][j].set_title(f'Min_dist = {min_dist}; n_neighbors = {n_neighbors}', fontsize = 12)

        print(min_dist, n_neighbors, 'complete!')

# add general label
plt.xlabel('UMAP 1', fontsize = 14)
plt.ylabel('UMAP 2', fontsize = 14)

# automatically make subplots fit together well
fig.tight_layout()

Putting name to clusters

In [ ]:
   
from scipy.stats import zscore

# Add cluster information to DataFrame
dftmp['CellType'] = clusters
cellsdf['CellType'] = clusters


# Drop the 'CellType' column before calculating means and plotting
features_df = dftmp.drop(columns=['CellType'])
cluster_means = features_df.groupby(dftmp['CellType']).mean()

# Identify markers that are positively expressed
positive_markers = cluster_means.map(lambda x: x if x > 0 else np.nan).dropna(axis=1, how='all')

# Calculate Z-scores for each feature in each cluster
z_scores = cluster_means.apply(zscore)

# List of thresholds for high expression
thresholds = {
    "75% confidence (Z > 0.675)": 0.675,
    "80% confidence (Z > 0.842)": 0.842,
    "85% confidence (Z > 1.036)": 1.036,
    "90% confidence (Z > 1.28)": 1.28
}

for confidence, threshold in thresholds.items():
    # Identify highly expressed markers for the current threshold
    highly_expressed_markers = z_scores.map(lambda x: x if x > threshold else np.nan).dropna(axis=1, how='all')

    # Print the result
    print(f"Highly expressed markers in each cell type ({confidence}):")
    for cluster in highly_expressed_markers.index:
        markers = highly_expressed_markers.loc[cluster].dropna().index.tolist()
        print(f"Highly expressed markers in cluster {cluster}:")
        for marker in markers:
            print(marker)
        print("\n")

# Plot the mean expression of features by cluster without the 'Cluster' column
if 'Cluster' in cluster_means.columns:
    cluster_means = cluster_means.drop(columns=['Cluster'])

ax = cluster_means.plot(kind='bar', figsize=(14, 7))
plt.title('Mean Expression of Features by Cluster')
plt.ylabel('Mean Expression')
plt.xlabel('Cluster')
plt.xticks(rotation=0)
plt.show()


**Cluster with labels**

In [ ]:

# Add cluster information to DataFrame
dftmp['CellType'] = clusters
cellsdf['CellType'] = clusters

# Generate UMAP embedding
embedder = UMAP(n_components=2, min_dist=0.76, n_neighbors=55).fit(dftmp)
embedding = embedder.transform(dftmp)

# Print the result
marker_labels = {}
print("Highly expressed markers in each cell type (90% confidence):")
for cluster in highly_expressed_markers.index:
    markers = highly_expressed_markers.loc[cluster].dropna().index.tolist()
    clean_markers = [marker.replace("Cell: ", "").replace(" mean", "").replace("Cluster", "") for marker in markers]
    label = f"Cluster {cluster}: " + ", ".join(clean_markers)
    marker_labels[cluster] = label
    print(label)
    print("\n")

# Plot UMAP projection
plt.figure(figsize=(10, 8))
sb.scatterplot(x=embedding[:, 0], y=embedding[:, 1], hue=dftmp['CellType'], palette='tab10')
for cluster in range(nclusters):
    label = marker_labels.get(cluster + 1, f"Cluster {cluster+1}")
    x = embedding[dftmp['CellType'] == (cluster+1), 0].mean()
    y = embedding[dftmp['CellType'] == (cluster+1), 1].mean()
    plt.text(x, y, label, fontsize=12, weight='bold')
plt.title('UMAP with Cluster Labels')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.show()


**Clusters over the image** 

In [ ]:
# Define specific colormaps for each cluster
cluster_colors = {
    1: 'Blues',
    2: 'Oranges',
    3: 'Greens',
    4: 'Reds',
    5: 'Purples'
}

# Generate hexbin plots for each cluster
fig, ax = plt.subplots(figsize=(10, 10))

# Plot each cluster with the respective colormap
for cluster, cmap in cluster_colors.items():
    cluster_data = cellsdf[cellsdf['CellType'] == cluster]
    hb = ax.hexbin(cluster_data['Xc'], cluster_data['Yc'], 
                   gridsize=50, cmap=plt.colormaps[cmap], 
                   mincnt=1, alpha=0.6, linewidths=0.5, edgecolors='k')

# Manually create a legend
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=plt.colormaps[cmap](0.6), markersize=10, label=f'Cluster {cluster}') for cluster, cmap in cluster_colors.items()]
ax.legend(handles=handles, title="Clusters")

# Add color bar and labels
cb = plt.colorbar(hb, label='Counts', orientation='vertical')
plt.title('Cluster Distribution')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.show()